In [4]:
import os
os.environ["OMP_NUM_THREADS"]='16'
import numpy as np
from scipy import linalg
from functools import reduce
np.set_printoptions(precision=2)
import matplotlib.pyplot as plt
import tensornetwork as tn
import time,math,gc,itertools
from scipy.sparse.linalg import LinearOperator
from scipy.sparse.linalg import eigsh
from scipy.sparse.linalg import svds
from colorsys import hls_to_rgb
from tqdm import tqdm
import opt_einsum
import sys
def show(z):
    if len(z.shape)==1:
        z=z.reshape((1,z.shape[0]))
    r,arg = np.abs(z),np.angle(z)
    h,s = (arg + np.pi)  / (2 * np.pi) + 0.5,0.8
    l=r**(1/2.2) #gamma correction
    l=l/(2*np.max(l))
    c = np.array(np.vectorize(hls_to_rgb) (h,l,s)).swapaxes(0,2).swapaxes(0,1)
    plt.imshow(c)
    plt.show()
def tensorDotAndReplace(tensor1, tensor2, axes1, axes2, replacedAxes):
    assert len(axes1)==len(replacedAxes) and len(axes2)==len(replacedAxes)
    first=np.delete(np.arange(tensor1.ndim),axes1)
    second=np.delete(np.arange(tensor2.ndim),axes2)
    k=tensor1.ndim
    for i in range(len(second)):
        flag=False
        for j in range(len(replacedAxes)):
            if second[i]==replacedAxes[j]:
                second[i]=axes1[j]
                flag=True
                break
        if not flag:
            second[i]=k
            k+=1
    rtval=np.tensordot(tensor1,tensor2,(axes1,axes2))
    rtval=rtval.transpose(np.argsort(np.concatenate((first,second))))
    #print(np.arange(tensor1.ndim), np.arange(tensor2.ndim), axes1, axes2, replacedAxes)
    #print(np.concatenate((first,second)))
    #print(np.argsort(np.concatenate((first,second))))
    return rtval
MEMORY_LIMIT=30*1024*1024*1024

In [5]:
import threading,time,datetime
import io,sys
from tqdm import tqdm
from IPython.display import clear_output

class BackgroundRunning:
    def __init__(self, job):
        self.result=None
        self.display=io.StringIO()
        self.job=job
    def start(self):
        self.start_time=time.time()
        def run_thread():
            self.result=job(self.display)
        self.thread=threading.Thread(target=run_thread,args=())
        self.thread.start()
                
    def showStatus(self):
        def sec2str(sec):
            return str(datetime.timedelta(seconds=int(sec)))
        
        clear_output(wait=True)
        elapsed_time=time.time()-self.start_time
        s='[{}]'.format(sec2str(elapsed_time))
        print(s)
        s=self.display.getvalue()
        x=s.rfind('\r')
        if x!=-1:
            s=s[x+1:]
        print(s)
        if self.is_done():
            print('done')
    def monitor(self,duration=10):
        for i in range(duration*4):
            self.showStatus()
            if self.is_done():
                return
            time.sleep(.25)
        print('to be continue...')
    def is_done(self):
        return not self.thread.is_alive()

In [6]:
def getCG(n):
    if n==0:
        return np.array([1])
    CG=np.zeros((n+1,)+(2,)*n)
    for i in range(2**n):
        indices=tuple(map(int,bin(i)[2:].zfill(n)))
        m=np.sum(indices)
        CG[(m,)+indices]=1
    CG=np.array(list(map(lambda x:x/linalg.norm(x),CG.reshape(n+1,-1)))).reshape(CG.shape)
    return CG
assert getCG(3).shape==(4,2,2,2)
CGs=[getCG(i) for i in range(9)]
singlet = np.array([[0,1],[-1,0]])
assert (singlet.dot(singlet)==-np.eye(2)).all()
assert (singlet==-singlet.T).all()
def getEdgeProjectorTensor(spin1,spin2):
    t1,t2,t3=CGs[spin1],CGs[spin2],CGs[spin1+spin2]
    t3=np.tensordot(t3,t1,([*range(1,spin1+1)],[*range(1,spin1+1)]))
    t3=np.tensordot(t3,t2,([*range(1,spin2+1)],[*range(1,spin2+1)]))
    return np.tensordot(t3,t3,(0,0))
edgeProjectorTensors=[[getEdgeProjectorTensor(i,j)for j in range(5)]for i in range(5)]

In [7]:
def contract_physical_edges(physical_edges1,physical_edges2):
    """
    Parameters
    ----------
    edges1,edges2:map vertice_id->edge
    """
    for k in physical_edges1:
        if k in physical_edges2:
            physical_edges1[k]^physical_edges2[k]
def contract_disconnected_tensor_network(nodes,edges):
    newNodes=[]
    nodes=set(nodes)
    while len(nodes)>0:
        newSet=set(tn.reachable(next(iter(nodes))))
        newEdges=list(filter(lambda x:x.get_nodes()[0] in newSet or x.get_nodes()[1] in newSet,edges))
        #print(newSet,[[x.get_nodes()[0],x.get_nodes()[1]] for x in newEdges])
        newNode=tn.contractors.auto(newSet,newEdges)
        nodes=nodes.difference(newSet)
        newNodes.append(newNode)
    result=tn.outer_product_final_nodes(newNodes,edges)
    return result 

In [8]:
assert sys.version_info>=(3,7) #will use dict ordering feature
class AKLT_Tensor:
    """
    Attributes
    ----------
    physical_edges:sorted dict vertice_id->edge
    virtual_edges:sorted dict vertice_id->edge
    nodes:sorted dict vertice_id->node or None after SVD
    physical_dimension:int
    virtual_dimension:int
    
    """
    def __init__(self,spins,conns,div):
        """
        Parameters
        ----------
        spins:[2,2,2,2]
            the spins for the vertices of the parent graph
        conns:[[0,1],[1,2],[2,3]]
            array of pair of vertice_ids for edges in the parent graph
        div:[1,2]
            the set of vertice_ids for vertices in the subgraph
        """        
        self.nodes=dict()
        self.physical_edges=dict()
        self.virtual_edges=dict()
        
        curIndice=[1]*len(spins)
        for vid in div:
            self.nodes[vid]=tn.Node(CGs[spins[vid]])# shape: [n+1]+[2]*n
        for vid1,vid2 in conns:
            if vid1 in div and vid2 in div:
                n1=self.nodes[vid1]
                n2=tn.Node(singlet)
                n3=self.nodes[vid2]

                edges=n1.get_all_edges()
                edges[curIndice[vid1]]=n2[1]
                n1=tn.contract(n1[curIndice[vid1]]^n2[0]).reorder_edges(edges)
                self.nodes[vid1]=n1

                n1[curIndice[vid1]]^n3[curIndice[vid2]]
                curIndice[vid1]+=1
                curIndice[vid2]+=1
        for vid in div:
            n1=self.nodes[vid]
            self.physical_edges[vid]=n1[0]
            num=spins[vid]+1-curIndice[vid]
            if num==0:
                pass
            elif num==1:
                self.virtual_edges[vid]=n1[curIndice[vid]]
            else:
                n2=tn.Node(CGs[num].conj())
                edges=n1.get_all_edges()[0:curIndice[vid]]+[n2[0]]
                for i in range(num):
                    n1[curIndice[vid]+i]^n2[1+i]
                n1=tn.contract_between(n1,n2,output_edge_order=edges)
                self.nodes[vid]=n1
                self.virtual_edges[vid]=edges[-1]
        
        self.nodes=dict(sorted(self.nodes.items()))
        self.physical_edges=dict(sorted(self.physical_edges.items()))
        self.virtual_edges=dict(sorted(self.virtual_edges.items()))
        
        self.physical_dimensions=[e.dimension for e in self.physical_edges.values()]
        self.virtual_dimensions=[e.dimension for e in self.virtual_edges.values()]
        self.physical_dimension=np.prod(self.physical_dimensions)
        self.virtual_dimension=np.prod(self.virtual_dimensions)
        

    
    def print_dimensions(self):
        print("node shapes:",[(i,n.shape) for i,n in self.nodes.items()])
        print("physical dimensions:",self.physical_dimensions)
        print("virtual dimensions:",self.virtual_dimensions)
        print("contracted size:",self.physical_dimension,"x",self.virtual_dimension,"=",self.physical_dimension*self.virtual_dimension)
    
    def print_topology(self):
        inm={n:i for i,n in self.nodes.items()}
        toPrint=[]
        for n in self.nodes.values():
            for e in n.edges:
                if e in self.physical_edges.values():
                    assert e.node2 is None
                    toPrint.append([inm[e.node1],'p',e.dimension])
                elif e in self.virtual_edges.values():
                    assert e.node2 is None
                    toPrint.append([inm[e.node1],'v',e.dimension])
                else:
                    toPrint.append([e.node1 and inm[e.node1],e.node2 and inm[e.node2],e.dimension])
        print(toPrint)
    def copy(self):
        node_dict,edge_dict=tn.copy(self.nodes.values())
        rtval=object.__new__(self.__class__)
        
        rtval.nodes={k:node_dict[v] for k,v in self.nodes.items()}
        rtval.physical_edges={k:edge_dict[v] for k,v in self.physical_edges.items()}
        rtval.virtual_edges={k:edge_dict[v] for k,v in self.virtual_edges.items()}
        rtval.physical_dimensions=self.physical_dimensions.copy()
        rtval.virtual_dimensions=self.virtual_dimensions.copy()
        rtval.physical_dimension=self.physical_dimension
        rtval.virtual_dimension=self.virtual_dimension
        
        return rtval
    def get_tensor(self):
        t=self.copy()
        return contract_disconnected_tensor_network(
            list(t.nodes.values()),
            list(t.physical_edges.values())+list(t.virtual_edges.values())
        ).tensor


In [9]:
class AKLT_Physical_Basis:
    def __init__(self,spins,conns,div,validate=True):
        """
        Parameters
        ----------
        spins:[2,2,2,2]
            the spins for the vertices of the parent graph
        conns:[[0,1],[1,2],[2,3]]
            array of pair of vertice_ids for edges in the parent graph
        div:[1,2]
            the set of vertice_ids for vertices in the subgraph
        """
        tensor=AKLT_Tensor(spins,conns,div)
        # assert tn.check_correct(tensor.nodes.values())==None wrong for non-connected graph

        
        t1,t2=tensor.copy(),tensor.copy()
        contract_physical_edges(t1.physical_edges,t2.physical_edges)
        mm=contract_disconnected_tensor_network(
            list(t1.nodes.values())+list(t2.nodes.values()),
            list(t1.virtual_edges.values())+list(t2.virtual_edges.values())
        ).tensor
        
        print("begin SVD for ",tensor.virtual_dimension," x ",tensor.virtual_dimension)
        s2,v=linalg.eigh(mm.reshape((tensor.virtual_dimension,tensor.virtual_dimension)))
        s=np.sqrt(s2)
        vh=v.conj().T
        assert np.allclose(v@vh,np.eye(v.shape[0]))
        assert np.allclose(vh@v,np.eye(vh.shape[0]))
        print("maxmin singular values: ",max(s),min(s)," total ",len(s))
        
        #new_tensor=v@np.diag(1/s)
        new_tensor=v@np.diag(1/s)@vh
        new_node=tn.Node(new_tensor.reshape(tensor.virtual_dimensions+[new_tensor.shape[1]]))
        
        for i,e in enumerate(tensor.virtual_edges.values()):
            new_node[i]^e
                
        self.all_nodes=[new_node]+list(tensor.nodes.values())
        self.physical_edges=tensor.physical_edges
        self.virtual_edge=new_node[-1]
        self.virtual_dimension=new_tensor.shape[1]
        self.physical_dimensions=tensor.physical_dimensions.copy()
        self.physical_dimension=tensor.physical_dimension
        self.singular_values=s
        self.v_tensor=v
        
        if validate:
            print('start validate')
            self.test_unitary()
            self.test_idempotent()
            print('validate passed')
        
    def print_dimensions(self):
        print("node shapes:",[n.shape for n in self.all_nodes])
        print("physical dimensions:",self.physical_dimensions)
        print("virtual dimension:",self.virtual_dimension)
        print("contracted size:",self.physical_dimension,"x",self.virtual_dimension,"=",self.physical_dimension*self.virtual_dimension)
    
    def copy(self):
        node_dict,edge_dict=tn.copy(self.all_nodes)
        rtval=object.__new__(self.__class__)
        
        rtval.all_nodes=[node_dict[v] for v in self.all_nodes]
        rtval.physical_edges={k:edge_dict[v] for k,v in self.physical_edges.items()}
        rtval.virtual_dimension=self.virtual_dimension
        rtval.virtual_edge=edge_dict[self.virtual_edge]
        rtval.physical_dimensions=self.physical_dimensions.copy()
        rtval.physical_dimension=self.physical_dimension
        rtval.singular_values=self.singular_values.copy()
        rtval.v_tensor=self.v_tensor.copy()
        
        return rtval
    def get_tensor(self):
        t=self.copy()
        return contract_disconnected_tensor_network(
            t.all_nodes,
            list(t.physical_edges.values())+[t.virtual_edge]
        ).tensor
    
    def test_unitary(self):
        a,b=self.copy(),self.copy()
        contract_physical_edges(a.physical_edges,b.physical_edges)
        t=contract_disconnected_tensor_network(
            a.all_nodes+b.all_nodes,
            [a.virtual_edge,b.virtual_edge]
        ).tensor
        assert np.allclose(t,np.eye(self.virtual_dimension))
    def test_idempotent(self):
        def E(v):
            n0=tn.Node(v.reshape(self.physical_dimensions))
            n0_physical_edges={k:n0[i] for i,k in enumerate(self.physical_edges)}
            a,b=self.copy(),self.copy()
            contract_physical_edges(n0_physical_edges,a.physical_edges)
            a.virtual_edge^b.virtual_edge
            nodes=a.all_nodes+b.all_nodes+[n0]
            edges=list(b.physical_edges.values())
            n1=contract_disconnected_tensor_network(nodes,edges)
            return n1.tensor.flatten()
        v=np.random.random(self.physical_dimension)
        assert np.allclose(E(v),E(E(v)))
        
a=AKLT_Physical_Basis([4,2],[[0,1]],[0,1])
b=AKLT_Tensor([4,2],[[0,1]],[0,1])
pd=a.physical_dimension
vd=a.virtual_dimension
ua=a.get_tensor().reshape(pd,-1)@a.v_tensor
sa=np.diag(a.singular_values)
va=a.v_tensor
tb=b.get_tensor().reshape(pd,-1)
assert np.allclose(ua@sa@va.T,tb)
assert np.allclose(va.T@va,np.eye(vd))
assert np.allclose(va@va.T,np.eye(vd))
assert np.allclose(ua.T@ua,np.eye(vd))
assert np.allclose(ua@ua.T,ua@ua.T@ua@ua.T)

a.test_unitary()
a.test_idempotent()


begin SVD for  8  x  8
maxmin singular values:  1.1180339887498951 0.8660254037844386  total  8
start validate
validate passed


In [10]:
# decorated diamond subsubpatch
# 216it [00:07, 27.94it/s] tol 0 gap 0.17064623273929694
spins=[2,2,2,2,4]
conns=[1,5,2,5,3,5,4,5]
conns=np.array(conns).reshape(-1,2)-1
weights=[1,1,1,1]

In [17]:
# decorated diamond subsubpatch 1/4
# 231it [00:09, 24.98it/s] tol 0 gap 0.04266155818482387
spins=[2,2,2,2,4]
conns=[1,5,2,5,3,5,4,5]
conns=np.array(conns).reshape(-1,2)-1
weights=[.25,.25,.25,.25]

In [12]:
0.04266155818482387*4

0.17064623273929547

In [26]:
# square4x4 projpart
spins=[4,4,4,4,4,4,4,4,4]
a=(-7+np.sqrt(118))/3
conns=[1,2,2,3,1,4,4,7, 4,5,5,6,7,8,8,9,2,5,5,8,3,6,6,9]
conns=np.array(conns).reshape(-1,2)-1
weights=[1,1,1,1,a,a,a,a,a,a,a,a]

In [27]:
dimP=np.prod(np.array(spins)+1)
print('dimP',dimP)

def AKLT_Hamiltonian(v):
    u=np.zeros(v.shape)
    v=np.reshape(v,np.array(spins)+1)
    for ii,(i,j) in enumerate(conns):
        p=edgeProjectorTensors[spins[i]][spins[j]]
        w=tensorDotAndReplace(v,p,(i,j),(0,1),(2,3)).reshape(-1)
        u+=w*weights[ii]
    return u

dimP 1953125


In [28]:
memory_limit=30*1024*1024*1024

patch=AKLT_Physical_Basis(spins,conns,range(len(spins)))

def AKLT_Ground_Projector(v):
    n0=tn.Node(v.reshape(patch.physical_dimensions))
    n0_edges={i:n0[i] for i in range(len(patch.physical_dimensions))}
    n1=patch.copy()
    n2=patch.copy()
    contract_physical_edges(n0_edges,n1.physical_edges)
    n1.virtual_edge^n2.virtual_edge
    nodes=[n0]+n1.all_nodes+n2.all_nodes
    edges=[n2.physical_edges[i] for i in range(len(patch.physical_dimensions))]
    n3=tn.contractors.auto(nodes,edges,memory_limit=memory_limit)
    return n3.tensor.reshape(-1)

v=np.ones(dimP)
v/=np.linalg.norm(v)

v1=AKLT_Ground_Projector(v)
v2=AKLT_Ground_Projector(v1)
assert not np.isclose(np.linalg.norm(v1),0)
assert np.allclose(v1,v2)
v3=v1/np.linalg.norm(v1)
v4=AKLT_Hamiltonian(v3)
assert np.isclose(np.linalg.norm(v4),0)

begin SVD for  1296  x  1296
maxmin singular values:  1.8625545224084419 0.36153923706730756  total  1296
start validate
validate passed


In [29]:
tol=0


def job(file):
    pbar=tqdm(file=file)
    def OP(v):
        pbar.update(1)
        v=AKLT_Hamiltonian(v)+np.sum(weights)*AKLT_Ground_Projector(v)
        gc.collect(2)
        return v
    op=LinearOperator(shape=(dimP,dimP),matvec=OP)
    eigenvalues, eigenvectors = eigsh(op, k=1,tol=tol,which='SA')
    pbar.close()
    return eigenvalues,eigenvectors

bg=BackgroundRunning(job)
bg.start()


In [ ]:

bg.monitor()
if(bg.is_done()):
    eigenvalues, eigenvectors=bg.result
    print('tol',tol,'gap',eigenvalues[0])

[0:29:56]
4176it [29:56,  2.47it/s]


In [15]:
bg.result

In [40]:
v=eigenvectors[:,0]
print(np.linalg.norm(v))
print(np.linalg.norm(AKLT_Hamiltonian(v)))
print(np.linalg.norm(AKLT_Ground_Projector(v)))
print(np.linalg.norm(v-AKLT_Ground_Projector(v)))

1.0000000000000024
1.0083483624418097e-15
1.0000000000000024
1.174400491344792e-15


In [ ]:
assert False

In [25]:
# old method
tol=0
num_of_eigen=16

pbar=tqdm()
def H(v):
    u=np.zeros(v.shape)
    v=np.reshape(v,np.array(spins)+1)
    for ii,(i,j) in enumerate(conns):
        p=edgeProjectorTensors[spins[i]][spins[j]]
        w=tensorDotAndReplace(v,p,(i,j),(0,1),(2,3)).reshape(-1)
        u+=w*weights[ii]
    pbar.update(1)
    return u
def OP(v):
    gc.collect(2)
    return H(v)-np.sum(weights)*v
    
dimP=np.prod(np.array(spins)+1)
op=LinearOperator(shape=(dimP,dimP),matvec=OP)
eigenvalues, eigenvectors = eigsh(op, k=num_of_eigen,tol=tol)
pbar.close()
print(eigenvalues)

1193it [00:35, 33.78it/s]

[-3.   -3.   -3.   -3.   -3.   -3.   -3.   -3.   -2.89 -2.89 -2.89 -2.89
 -2.89 -2.89 -2.89 -2.89]


In [27]:
print(eigenvalues[8]-eigenvalues[7])

0.10808611471802587
